In [68]:
import numpy 
import pandas as pd
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")
df=movies.merge(credits,on='title')

In [69]:
ignore_cols = ["budget", "homepage", "original_language", "original_title",
               "production_companies", "runtime", "spoken_languages", 
               "release_date", "status", "tagline","movie_id","production_countries"]


In [70]:
df=df.drop(columns=ignore_cols, inplace=False)

In [71]:
df.head(3)

,genres,id,keywords,overview,popularity,revenue,title,vote_average,vote_count,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",150.437577,2787965087,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",139.082615,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,107.376788,880674609,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [72]:
df.dropna(inplace=True)

In [73]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
def cast_to_list(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter<5:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L
    

In [74]:
df["genres"]=df["genres"].apply(convert)
df["keywords"]=df["keywords"].apply(convert)
df["cast"]=df["cast"].apply(cast_to_list)


In [75]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [76]:
df["director"]=df["crew"].apply(fetch_director)

In [78]:
df.drop(columns=["crew"], inplace=True)

In [82]:
df["overview"]=df["overview"].apply(lambda x:x.split())

In [84]:
df["genres"]=df["genres"].apply(lambda x:[i.replace(" ","") for i in x])

In [86]:
df["cast"]=df["cast"].apply(lambda x:[i.replace(" ","") for i in x])
df["keywords"]=df["keywords"].apply(lambda x:[i.replace(" ","") for i in x])
df["director"]=df["director"].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
df["tags"]=df["overview"]+df["genres"]+df["keywords"]+df["cast"]+df["director"]

In [91]:
df.drop(columns=["overview","genres","keywords","cast","director"], inplace=True)

In [93]:
df["tags"]=df["tags"].apply(lambda x:" ".join(x))

In [95]:
new_df=df

In [97]:
new_df.to_csv("final_movie_data.csv",index=False)

In [98]:
df=pd.read_csv("final_movie_data.csv")

In [99]:
df.head()

,id,popularity,revenue,title,vote_average,vote_count,tags
0,19995,150.437577,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di..."
1,285,139.082615,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha..."
2,206647,107.376788,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...
3,49026,112.312950,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...
4,49529,43.926995,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca..."


#tags preprocess

In [94]:
# tags preprocess
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(df["tags"]).toarray()
vectors.shape


(4806, 5000)